In [4]:
from z3 import *

A = DeclareSort("A")
a,b,c,d,e = Consts("a b c d e", A)
s = Solver()
s.add(a == b)
s.add(b == c)
s.add(d == e)


s.check()
s.model()



[e = A!val!0,
 d = A!val!0,
 c = A!val!0,
 a = A!val!0,
 b = A!val!0]

The problem is that the model has all the elements equal to each other.


In [7]:
from z3 import *

A = DeclareSort("A")
EClass = DeclareSort("Eclass")
a,b,c,d,e = Consts("a b c d e", A)
find = Function("find", A, EClass)

s = Solver()
s.add(Distinct(a,b,c,d,e)) # or use ADT enum
s.add(find(a) == find(b))
s.add(find(b) == find(c))
s.add(find(d) == find(e))


s.check()
s.model()
# huh this kind of worked...

[b = A!val!1,
 a = A!val!0,
 c = A!val!2,
 d = A!val!3,
 e = A!val!4,
 find = [A!val!3 -> Eclass!val!1,
         A!val!4 -> Eclass!val!1,
         else -> Eclass!val!0]]

In [ ]:
from z3 import *

A = DeclareSort("A")
EClass = DeclareSort("Eclass")
a,b,c,d,e = Consts("a b c d e", A)
find = Function("find", A, EClass)
root = Function("root", EClass, A)

s = Solver()
s.add(Distinct(a,b,c,d,e)) # or use ADT enum
s.add(find(a) == find(b))
s.add(find(b) == find(c))
s.add(find(d) == find(e))
#s.add(ForAll([x,y], (root(find(x)) == find(y)) == 
#    Or(And(Path.is_refl(p), x == y),
#       And(Path.is_trans(p), And(find(z) == find(x), find(z) == find    
#    ))
s.add(ForAll([x,y,p], (root(find(x)) == y) == 
    Or(And(Path.is_refl(p), x == y),
       And(Path.is_trans(p), And(find(Path.z(p) == find(x), find(z) == find    
    ))

Path = Datatype("Path")
Path.declare("refl", ("x", A))
Path.declare("trans", ("x", A), ("cdr", Path))
Path = Path.create()

rootpath = Function("rootpath", A, Path)
root = Function("root", Path, A)
root(p) == If(Path.is_refl(p), Path.x(p),  root(Path.cdr(p)))


ForAll([x], 
Or(
    Path.is_refl(rootpath(x)),
    And(Path.is_trans(x),
        Or(And(x == a, Path.x(rootpath(x)) == b),
            And(x == b, Path.x(rootpath(x)) == c)
    )
    )



s.check()
s.model()




The thng I was trying to do was embed egglog / equality sauratin in z3. 
Z3 already has an egraph. It already has an ematcher. The problem is that the semantics of equality saturation is to return the least model. This is typical. We can solve it through the generic method of pwel founded proof objects.

A tricky thing is that the intrinsic (intensional?) equality is too powrful. It would seemingly make things equal out from under our feet.

A trick we encounered in egglog is to make copies of your synta.

Another trick is a rtatified set of equalities, each one a refinment of the other.

Another trick is proof objects. We can have proof objects denote somethng.

```python
Term = Datatype("Term")
Term.declare("lam", ("x", IntSort()), ("body", ))

def denote(ty):
    def res(t):
        den = Function(f"denote_{ty}", Term, ty)
        return den(t)
    def res

```







# Old
Something I like about datalog and ASP is that their every result is justified.
The models output by an smt or sat solver are kind of more random.

This is related to the idea that you can't in general express the transitive closure of a relation in first order logic. "Whhhaaaaa?" you say. I agree. It feels odd.

Doesn't this work?
`forall x y z, R(x,y) & Rstar(y,z) => Rstar(x,z)`
`forall x y, R(x,y) => Rstar(x,y)`

No, because these axioms actually don't precisely constrain `Rstar` to precisely the transitive closure. They constraint `Rstar` to be _at least_ the transitive closure, which is a little different. For example, `[R(x,y) = false, Rstar(x,y) = true]` is a model of these axioms.

Okay, well what about if we try and make the thing bidirectional.

`forall x,y,z Rstar(x,z) <=> R(x,y) & Rstar(y,z) | R(x,z)` This is a little better. However, it doesn't account for self consistent / non well founded loops in the model.
Consider `[R(1,2) = true,  Rstar(1,2) = true, Rstar(2,1) = true]`. This is a tiny 2 vertex loop. It is a model of the above axioms, but still `Rstar` is not the transitive closure of R.

There are some doges for this problem. If R is for serious a particular finite relation, I can precompute the transitive clousure and just write that to constraint Rstar. Or more generally, if R is of bounded size, I could precompute all transitive closures or break all possible loops with axioms.

Answer Set programming could be called ["justified smt"](https://www.weaselhat.com/2022/11/07/asp/).  
However, Z3 does have a notion of well foundedness/least fixed point, that of Algebraic data types.

Another notion of least comes from using the optimization capabilities of z3. If we try to minimize the number of places `Rstar` is true, we'll get the minimal model and only get the pieces that are _forced_ to be true by the axioms. This is the transitive closure.

_Also_, z3 has the notion of transitive closure built in a as special relation.

I realized this trying to use Z3's egraph in a justified way in which it produces the minimal model. The same problem happens. If I say `a = b /\ b = c /\ d = e` what I want `=` to do is be exactly the symmetric transitive clousure of the axioms I said. It is not however, instead collapsing into a single object in the model `[a = V1, b = V1, c = V1, d = V1, e = V1]` is valid.

Another way of viewing `=` is trying to talk about connectivity in an undirected graph. The "proof" of two things being connected is a path between them that starts on one vertex and ends on the other.

Internalizing proofs as objects is quite a fun brain tickling idea. It is an old one too. We are playing godelian games.

Here's another example: parsing. Parsing as deduction. The existence of the parse tree itself is a proof that such and such a string does parse / belongs to the CFG defined set of strings. <https://lexi-lambda.github.io/blog/2019/11/05/parse-don-t-validate/> . Indeed,  parse trees make invalid strings unrepresentable.

Hmm. The well foundedness of strings themselves might save me here.

parsing

```python
from z3 import *
sexp = Function("sexp", StringSort(), BoolSort())
s.add(ForAll([s], 
    sexp(s) == Or(s == "()", And(s[0] == "(", sexp(s[1:-1]), s[-1] == ")"))))

```

quoting


## Datalog

### uZ3

```z3
;re

(declare-sort A)

(declare-rel add (A A A))
(declare-var a A)
(declare-var b A)
(declare-var c A)
(declare-const x1 A)
(declare-const x2 A)
(declare-const x3 A)

(rule (=> (add a b c) (add b a c)))
(rule (add x1 x2 x3))

(query add :print-answer true)


```

```z3
;(set-option :fixedpoint.engine datalog)
(define-sort s () (_ BitVec 3))
(declare-rel edge (s s))
(declare-rel path (s s))
(declare-var a s)
(declare-var b s)
(declare-var c s)

(rule (=> (edge a b) (path a b)))
(rule (=> (and (path a b) (path b c)) (path a c)))

(rule (edge #b001 #b010))
(rule (edge #b001 #b011))
(rule (edge #b010 #b100))

(declare-rel q1 ())
(declare-rel q2 ())
(declare-rel q3 (s))
(rule (=> (path #b001 #b100) q1))
(rule (=> (path #b011 #b100) q2))
(rule (=> (path #b001 b) (q3 b)))

(query q1)
(query q2)
(query q3 :print-answer true)
```

### Clark Completion

[clark completion](https://www.inf.ed.ac.uk/teaching/courses/lp/2012/slides/lpTheory8.pdf)
Only things that are forced to be true are true.
Take horn clause  head(x) :- b(x).
Normalize by putting equalities in the body. This means ground facts become. `edge(1,2). ---> edge(X,Y) :- X = 1, Y = 2.`
Now collect up all rules with heads. head == body1 \/ body2 \/ body3 \/

This example may be deceptive.
`p :- p.` is an example rule that in clark completion allows self reinforcing of p. It justifies itself. Not good.
So perhaps one way of thinking about this is you need some kind of termination argument, inductive type, or minimality restriction.
This particular example got lucky

```z3
(declare-fun edge (Int Int) Bool)
(declare-fun path (Int Int) Bool)
(assert (forall ((x Int) (y Int))
  (= (edge x y)
     (or  (and (= x 1) (= y 2))
          (and (= x 2) (= y 3))
          (and (= x 3) (= y 4)))
)))

(assert (forall ((x Int) (y Int))
  (= (path x y)
     (or
          (edge x y)
          (exists ((z Int)) (and (edge x z) (path z y))))
)))
(check-sat)
(get-model)
```

Closed world assumption. I guess that's simpler

### Transitive Closure

[relational constraint solving in smt](https://homepage.divms.uiowa.edu/~ajreynol/cade17.pdf)
<https://cvc5.github.io/docs/cvc5-1.0.0/theories/sets-and-relations.html>

special relations. Transitive closure cannot be expressed in first order logic
<https://math.stackexchange.com/questions/295298/is-reflexive-transitive-closure-of-relation-r-a-first-order-property>

My gut says this is a pretty nuanced statement.

Simple obvious axioms actually express that it is an overapproximation of the theory. For example R(a,b) = True is a valid model of the transtivieity axioms

[Some Remarks on the Definability of Transitive Closure in First-order Logic and Datalog](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.127.8266&rep=rep1&type=pdf)

<https://microsoft.github.io/z3guide/docs/theories/Special%20Relations#transitive-closures>

expansion operator of database. This is a little nuts
{(a,b), (b,c)} R  (a,b)

explicit

deterministic transitive closure. Transitive closure of a functional relationship is a little bit lesser.
<http://cs.technion.ac.il/~shachari/dl/thesis.pdf> tricking your way around these restrictions. Kind of using an indirect minimality condition.

### Explicit datalog iteration

```
(define-fun path (Int Vert Vert) Bool)

;(define-fun-rec )
(forall ((iter Int) (x Vert) (y Vert))
  (=> (and (< 0 i) (<= i iterlim))
     (= (path (+ 1 iter) x y)
        (or (path iter x y)
            ; clauses.
        )
     )
)

(forall x y
 (path 0
)


```

We can check if `(assert (!= (path N x y) (path (+ N 1) x y))` for whether we've terminated or not.

### Well founded

`Option Bool` is also acceptable

```z3
(declare-datatype WF (
  (ProveTrue) (ProveFalse) (Unknown)
))

(define-fun <== ((x WF) (y WF)) Bool
  (match x (
    (KnownTrue)
    (KnownFalse)
    (Unknown)
  )
  
  )
)

(define-fun not_ ((x WF)) Bool

)

```

This doesn't really help. Well founded semantics still has a minimality condition

### Proof Relevant

```z3
;re
(declare-datatype Vert (
  (a)
  (b)
  (c)
))
; datatype is much faster. Interesting.
;(declare-sort Vert)
;(declare-const a Vert)
;(declare-const b Vert)
;(declare-const c Vert)
;(assert (distinct a b c))
(declare-datatype Proof (
  (Edge)
  (Trans (trans-y Vert) (trans-pf Proof))
  )
)
(declare-datatype Option (
  par (T) (
    (None)
    (Some (get-some T))
  )
))



(define-fun edge ((x Vert) (y Vert)) Bool
  (or
    (and (= x a) (= y b))
    (and (= x b) (= y c))
  )
)

(declare-fun path (Vert Vert) (Option Proof))

; forward inference
(assert
  (forall ((x Vert) (y Vert))
    (=> (edge x y) (distinct None (path x y)))
  )
)

(assert
  (forall ((x Vert) (y Vert) (z Vert))
    (=> (and (edge x y) (distinct None (path y z))) (distinct None (path x z)))
  )
)
; backward direction. Every relation that is not None is justified.
(assert
  (forall ((x Vert) (y Vert))
    (=> (= (Some Edge) (path x y)) (edge x y))
  )
)

(assert
  (forall ((x Vert) (y Vert) (z Vert) (pf Proof))
    (=> (= (Some (Trans y pf)) (path x z)) 
        (and (edge x y) (= (Some pf) (path y z))))
  )
)

; alternative formulation. Meh.
;(assert
;  (forall ((x Vert) (y Vert))
;    (match (path x y)
;    (
;      (None true)
;      ((Some Edge) (edge x y))
;      ((Some (Trans z pf)) (and (edge x z) (= (Some pf) (path z y))))
;    )
;)))


(check-sat)
(get-model)
(eval (path a b))
(eval (path a a))
(eval (path a c))
(eval (path c a))
(eval (path b c))
```

In some sense, inductive datatypes are also a minimal fixed point. It seems that by making proof relevant relations, we can inherit this minimality?

One might thing you could remove the arguments to Trans. You can't remove `pf`. That is what is guaranteeing well foundedness of the derivation / acyclicity.

See also "Proof Objects" below.

```z3

;re
(declare-datatype Vert (
  (a)
  (b)
  (c)
))
(declare-datatype Proof (
  (Edge)
  (Trans (trans-y Vert) (level Int))
  )
)

(assert 
  (= (level Edge) 0)
)

(declare-datatype Option (
  par (T) (
    (None)
    (Some (get-some T))
  )
))



(define-fun edge ((x Vert) (y Vert)) Bool
  (or
    (and (= x a) (= y b))
    (and (= x b) (= y c))
  )
)

(declare-fun path (Vert Vert) (Option Proof))


(forall ((x Vert) (y Vert) (g Int))
 (=> (= (Some g) (path x y)) (>= (level g) 0))
)


; forward direction
(assert
  (forall ((x Vert) (y Vert))
    (=> (edge x y) (distinct None (path x y)))
  )
)

(assert
  (forall ((x Vert) (y Vert) (z Vert))
    (=> (and (edge x y) (distinct None (path y z))) (distinct None (path x z)))
  )
)
; backward direction. Every relation that is not None is justified.
(assert
  (forall ((x Vert) (y Vert))
    (=> (= (Some Edge) (path x y)) (edge x y))
  )
)

(assert
  (forall ((x Vert) (y Vert) (z Vert) (n Int) (pf Proof))
    (=> (= (Some (Trans y n)) (path x z)) 
        (and (edge x y) (= (level pf) (- n 1)) (= (Some pf) (path y z))))
  )
)

(check-sat)
(get-model)
(eval (path a b))
(eval (path a a))
(eval (path a c))
(eval (path c a))
(eval (path b c))
```

Proof semantics / Provenance
Do we cast to bool or not?
There is a proof relevant thing that tells you which case of the `or` you're in in the completion semantics.
I need negation because I need == and !=?

```
(declare-datatype Proof
  (Trans (trans1 Int))
  (Base)
  (Unknown)
)
(define and_ ((x Proof) (y Proof))  Bool
    (not (or (is-Uknown x) (is-Uknown y))
)

(assert
  (forall ((x Int) (y Int) (z Int)))
   (=> (= (path x z) (Trans y))
       (and_ (edge x y) (path y z))
   )
)


(assert
  (forall ((x Int) (y Int) (z Int)))
   (=> (= (path x z) (Trans y p q))
       (and (= p (edge x y)) (= q (path y z)))
   )
)

(assert 
(=> (= (path x y) (Base)))
    (not (is-Unknown (edge x y))
)

(assert 
  (=> (edge x y) (path x y)
)

(assert 
  (=> (and_ (edge x y) (path y z))
      (path )
  )
)
)

```

#### First Class Union Find

Hmm. Making an adt for spanning trees is difficult and annoying. Maybe using the notion of
x -> zipper. Because funion, it has unique spot in tree.

Is this n*2 space though? with sharing it shouldn't be
Using an order to tie break?
a -> Int
explain -> Explain i, reason

binary tree. Ordered.

SpanTree.

root(x) = root(y)
explain(x) = path between root(x) and x. = Trans(y,just_xy, pf)

```z3
;re
(declare-datatype Vert (
  (a) (b) (c) (d)))

(declare-datatype Path (
  (Refl)
  (Trans (y Vert) (trans-pf Path))
))
(declare-fun root (Vert) Vert)
(declare-fun explain (Vert) Path)

(define-fun edge0 ((x Vert) (y Vert)) Bool
  (or 
    ;(and (= x a) (= y b))
    (and (= x c) (= y d))
    (and (= x c) (= y a))
  )
)
(define-fun edge ((x Vert) (y Vert)) Bool
  (or (edge0 x y) (edge0 y x)))

; forward inference
(assert (forall ((x Vert) (y Vert))
      (=> (edge x y) (= (root x) (root y)))
))

; backwards justification
(assert (forall ((x Vert))
  (=> (= (explain x) Refl) (= (root x) x))
))

(assert (forall ((x Vert) (y Vert) (pf Path))
  (=> (= (explain x) (Trans y pf))
      (and (= (root x) (root y)) 
           (= pf (explain y)) ; optional? This is what makes it a spanning tree
           (edge x y)))))

(check-sat)
(get-model)


```

```z3
;re
(declare-datatype SpanTree (
  ((x Vert) ()
)
)

(declare-datatype Zipper
  (Zippper (List ))
)
(define-fun-rec in-tree ((x Int) (t Tree))
  (match t (

  ))
)
(define-fun-rec all-jsutified (t Tree)

)
(declare-fun component (Int) SpanTree)
(assert
  (= (comp a) (comp b))
)
(assert
  (= (comp b) (comp c))
)

(assert
    (forall ((x Vert)) (and (in-tree (comp x)) (all-jusitified (comp x)))
)


(declare-fun parent (Vert) (Option Vert)) ; but acuyclic.
(declare-fun parent (Vert) (Option SpanTree))
(declare-fun root (Vert) SpanTree)
(declare-fun root (Vert) Vert)
(declare-fun explain (Vert))


```

## Rewriting

z3 `simplify` command. This only simplifies under the base theories though.

```z3
(declare-sort aexpr)
(declare-fun add (aexpr aexpr aexpr) Bool)
(declare-fun fadd (aexpr aexpr) aexpr)

(declare-const x aexpr)
(declare-const y aexpr)
(declare-const z aexpr)

(assert (add x y z))

; reflection / functional dependency
; (add x y (fadd x y))  is slightly different. This generates all fadd.
(assert 
  (forall ((x aexpr) (y aexpr) (z aexpr)) (=> (add x y z)  (= z (fadd x y) ) ))
)

; clark completion?
(assert 
  (forall ((x aexpr) (y aexpr) (z aexpr))  (= (add x y z) (add y x z)))
)

; we need something that says two aexpr are only equal if forced to be. There exists a reason.
; clark completion works on atoms
(= (= x y) (exists z (add x y z) ())


; functional dependency
; using implication. fishy?
;(assert 
;  (forall ((x aexpr) (y aexpr) (z aexpr) (z1 aexpr))  (=> (and (add x y z) (add x y z1))) (= z z1))
;)


(check-sat)
(get-model)


```

```z3
(declare-sort aexpr)
(declare-fun add (aexpr aexpr aexpr) Bool)
(declare-fun fadd (aexpr aexpr) aexpr)

(declare-const x aexpr)
(declare-const y aexpr)
(declare-const z aexpr)

(assert (add x y z))

; reflection / functional dependency
; (add x y (fadd x y))  is slightly different. This generates all fadd.
(assert 
  (forall ((x aexpr) (y aexpr) (z aexpr)) (=> (add x y z)  (= z (fadd x y) ) ))
)

; clark completion?
(assert 
  (forall ((x aexpr) (y aexpr) (z aexpr))  (= (add x y z) (add y x z)))
)




(check-sat)
(get-model)


```

(not (= x y)) iff no functional dependency violation.

forall z1, z2, (not (= z1 z2)) iff (not (exists (x y) (add x y z1) (add x y z2)))
what about simple constants
(X x) (Y y) => x != y all pairs of constant tables?
But why?

forall z1, z2. z1 != z2 <->  (not (add(x,y,z1) /\ add(x,y,z2)) )
forall z1, z2. z1 != z2 <->  (not (mul(x,y,z1) /\ mul(x,y,z2)) )
forall z1, z2. z1 != z2 <->  (not (add(x,y,z1) /\ add(x,y,z2)) )
forall z1, z2. z1 != z2 <->  (not (X z1 /\ X z2) )

// No but this isn't right. We _might_ have these equal
forall z1, z2. Y z1 /\ X z2 -> z1 != z2  // all pairs
forall z1, z2. Y z1 /\ Z z2 -> z1 != z2

Yes some are existential head rules. hmm.
exists z, X z.
where does the exists go?
Skolemize them I guess.
forall x y, head(x, y) <-> \/  ( y = skolem(x) /\ body(x)  )

so
exists z, X z. becomes
exists z, forall x, X x <-> x = z.

exists z, f(y,z) :- f(x,y)

f(a).
f(f(X)) <- f(X).

forall z y, f(y,z) = (z = f(y) /\ exists x, f(x,y)  \/  f(a,fa)   )

algerbaic datatypes + datalog have the same problem

Maybe if I know the big terms that are causing nontermination, I could cap them. Over approximate equality.
Checking two terms can be done as a separate smt query.

((depth (num 1)) = 1)

(depth (add x y)) <= 1 + max(depth(x),depth(y))

mbqi is a kind of like datalog isn't it?

If we want things bounded, perhaps explicilty bound the state of eids?
n : _ BitVec 20
(eid n)

Logically there is nothing forcing it to have 3 universe elements here. Hmm. We need equality to be forced by some reason. clark semantics on the equality relation. (= x y) =

(eid n x) as a relation (eid bitvec aexpr)

bounded skolem function?

We could explicitly use three valued semantics. What about encoding an ASP query to smt?

= is explicitly a relation. We can't quantify the thing?
One reason is refl. But we can't really talk about refl
So we need to expand the forall and exlude the refl cases, which don't need to be dignified.
(= (= a b)  ((a = c) /\ (= b c)
(skolem x) = a

(= (= x y) (or (((exists a b) (= (add a b) x) and (= (add a b) y)
            ((   exists a b     (= (add a b) x) (= (add b a) y)  ))

            )
add(a,b) = add(b,a) if

(add a b c)
(=

)

(= (baseeq x y) (or (= x y)
                    (and (add a b x) (and a b y))
)

(= (_ transitive baseq)  (a = b)